In this lesson, you will learn about the six key elements which help make Agents perform even better:

Role Playing
Focus
Tools
Cooperation
Guardrails
Memory

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

Role Playing, Focus and Cooperation

In [4]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

"""By not setting allow_delegation=False, allow_delegation takes its default value of being True.
This means the agent can delegate its work to another agent which is better suited to do a particular task."""

'By not setting allow_delegation=False, allow_delegation takes its default value of being True.\nThis means the agent can delegate its work to another agent which is better suited to do a particular task.'

In [5]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

"""Role Playing: Both agents have been given a role, goal and backstory.
Focus: Both agents have been prompted to get into the character of the roles they are playing.
Cooperation: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together."""

'Role Playing: Both agents have been given a role, goal and backstory.\nFocus: Both agents have been prompted to get into the character of the roles they are playing.\nCooperation: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.'

Tools, Guardrails and Memory

In [6]:
#Import CrewAI tools

from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

"""Possible Custom Tools
Load customer data
Tap into previous conversations
Load data from a CRM
Checking existing bug reports
Checking existing feature requests
Checking ongoing tickets
... and more

#Some ways of using CrewAI tools

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
"""

'Possible Custom Tools\nLoad customer data\nTap into previous conversations\nLoad data from a CRM\nChecking existing bug reports\nChecking existing feature requests\nChecking ongoing tickets\n... and more\n\n#Some ways of using CrewAI tools\n\nsearch_tool = SerperDevTool()\nscrape_tool = ScrapeWebsiteTool()\n'

In [7]:
#Instantiate a document scraper tool.
#The tool will scrape a page (only 1 URL) of the CrewAI documentation.

docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

"""Different Ways to Give Agents Tools
Agent Level: The Agent can use the Tool(s) on any Task it performs.
Task Level: The Agent will only use the Tool(s) when performing that specific Task.
Note: Task Tools override the Agent Tools."""

'Different Ways to Give Agents Tools\nAgent Level: The Agent can use the Tool(s) on any Task it performs.\nTask Level: The Agent will only use the Tool(s) when performing that specific Task.\nNote: Task Tools override the Agent Tools.'

Creating Tasks

In [8]:
#You are passing the Tool on the Task Level.

inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [9]:
#quality_assurance_review is not using any Tool(s)
#Here the QA Agent will only review the work of the Support Agent

quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


Creating the Crew

In [10]:
#Memory: Setting `memory=True` when putting the crew together enables Memory.

crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

Running the Crew: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
#Guardrails: By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to provide the best support to DeepLearningAI by thoroughly understanding their question and providing a detailed and informative response.

Action: Read website content
Action Input: {} 

Assembling and Activating Your CrewAI Team - crewAI
 Skip to content
 crewAI
 Assembling and Activating Your CrewAI Team
 Initializing search
 crewAI
 crewAI
 crewAI
 Home
 Core Concepts
 Core Concepts
 Agents
 Tasks
 Tools
 Processes
 Crews
 Collaboration
 Memory
 U

In [12]:
# Display the final result as Markdown

from IPython.display import Markdown
Markdown(result)

Hello Andrew Ng,

Thank you for reaching out to crewAI for guidance on adding memory to your crew. I'm excited to assist you in this task and ensure you have a smooth experience with CrewAI.

To add memory to your crew in CrewAI, you can follow the steps outlined below:

1. Assemble Your Agents: Start by defining your agents with unique roles, backstories, and enhanced capabilities like memory usage. By setting the memory parameter to True when defining the agent, you can create agents with memory capabilities. Here is an example code snippet to guide you:
```
researcher = Agent(
    role='Senior Researcher',
    goal='Uncover groundbreaking technologies in {topic}',
    verbose=True,
    memory=True,
    backstory=(
        "Driven by curiosity, you're at the forefront of innovation, eager to explore and share knowledge that could change the world."
    ),
    tools=[search_tool, browser_tool],
)
```

2. Form the Crew: Combine your agents into a crew and establish the workflow process they will follow to complete tasks. To enable memory for the crew, set the memory parameter to True. Here is an example code snippet for forming the crew:
```
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_task],
    process=Process.sequential, # Optional: Sequential task execution is default
    memory=True,
    cache=True,
    max_rpm=100,
    manager_agent=manager
)
```

3. Kick It Off: Initiate the process with your enhanced crew ready. Ensure that memory is enabled for the crew to utilize memory capabilities during task execution. Here is an example code snippet to get you started:
```
result = crew.kickoff(inputs={'topic': 'AI in healthcare'})
```

By following these steps, you will be able to successfully add memory to your crew in CrewAI and leverage memory capabilities for efficient task execution and collaboration within your AI team.

If you have any further questions or need additional assistance, please feel free to reach out to us. We are committed to helping you make the most out of your CrewAI experience and ensure your success.

Best regards,
[Your Name]
Senior Support Representative
crewAI